In [ ]:
using Flux
using Distributions
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated
# using CUDAapi

In [13]:

using StatsBase

#a model is a struct of a name and a function which takes an experiment and outputs an observation
struct Model
  name
  func
end
(m::Model)(u) = m.func(u)
#the prior, priorsamp is a function which returns a random model according to the prior dist
#rejection sampling event with probability 0
struct ZeroProb <: Exception
end

function postsamp(priorsamp,exp,obs)
  testmodel = priorsamp()
  testobs = testmodel(exp)
  counter = 0
  while counter < 100000 && !(testobs == obs)
    counter += 1
    testmodel = priorsamp()
    testobs = testmodel(exp)
  end
  if counter == 100000
    throw(ZeroProb())
  end
  testmodel
end

function countdict(sampler,n_samps)
  counts = Dict()
  for iter in 1:n_samps
    model=sampler()
    if !haskey(counts,model.name)
      counts[model.name] = 1/n_samps
    else
      counts[model.name] += 1/n_samps
    end
  end
  counts
end


KL_sample_size = 100000
function approxKL(dist1,dist2)
  #firstsamples = [dist1() for i in 1:KL_sample_size]
  #secondsamples = [dist2() for i in 1:KL_sample_size]
  countdict1 = countdict(dist1, KL_sample_size)
  countdict2 = countdict(dist2, KL_sample_size)
  total = 0
  for (key,val) in countdict1
    if val > 0
      total += val * (log(val) - log(countdict2[key]))
    end
  end
  total
end

util_sample_size = 1000
function approxutility(exp,priorsamp)
  KLdict = Dict()
  total = 0
  for i in 1:util_sample_size
    data = priorsamp()(exp)
    if haskey(KLdict,data)
      total += KLdict[data]
    else
      postdist = () -> postsamp(priorsamp,exp,data)
      div = approxKL(postdist,priorsamp)
      total += div
      KLdict[data] = div
    end
    println(i)
  end
  total / util_sample_size
end
#here we assume finite experiment set--how to maximize over infinite experiment
#set without any structure on the experiments?
function optimalexp(priorsamp,experiments)
  max = 0
  bestexp = -1
  for exp in experiments
    u = approxutility(exp, priorsamp)
    if u >= max
      max = u
      bestexp = exp
    end
  end
  bestexp
end

function exactpost(prior,likelihoods,exp,data)
  dataprob = sum([prior[i] * likelihoods[i](exp,data) for i in 1:length(prior)])
  [prior[i] * likelihoods[i](exp,data) / dataprob for i in 1:length(prior)]
end

function exactKL(probs1, probs2)
  sum([probs1[i] * (log(probs1[i] / probs2[i])) for i in 1:length(probs1)])
end

function analyticutil(prior,priorsamp,likelihoods,exp,samples)
  KLdict = Dict()
  total = 0
  for i in 1:samples
    data = priorsamp()(exp)
    if haskey(KLdict,data)
      total += KLdict[data]
    else
      postprobs = exactpost(prior,likelihoods,exp,data)
      div = exactKL(postprobs,prior)
      total += div
      KLdict[data] = div
    end
  end
  total / samples
end

function analyticoptimalexp(prior,priorsamp,likelihoods,experiments,samples)
  max = 0
  bestexp = -1
  for exp in experiments
    u = analyticutil(prior,priorsamp,likelihoods,exp,samples)
    #println(exp,u)
    if u >= max
      max = u
      bestexp = exp
    end
  end
  bestexp
end


n_exps = 1
function biasedfunc(exp,n)
  if sum(exp) == 4
    phead = 5/6
  elseif sum(exp) == 3
    phead = 2/3
  elseif sum(exp) == 2
    phead = 1/2
  elseif sum(exp) == 1
    phead = 1/3
  else
    phead = 1/6
  end
  [Int(rand() < phead) for i in 1:n]
end
m_biased = Model("biased",(exp) -> biasedfunc(exp,n_exps))

function markovfunc(exp,n)
  transitions = 0
  for i in 1:3
    if exp[i] != exp[i+1]
      transitions += 1
    end
  end
  if transitions == 3
    t_prob = 4/5
  elseif transitions == 2
    t_prob = 3/5
  elseif transitions == 1
    t_prob = 2/5
  elseif transitions == 0
    t_prob = 1/5
  end
  if rand() < t_prob
    1 - exp[4]
  else
    exp[4]
  end
  vals=[]
  for i in 1:n
    if rand() < t_prob
      push!(vals, 1 - exp[4])
    else
      push!(vals, exp[4])
    end
  end
  vals
end
m_markov = Model("markov",(exp) -> markovfunc(exp,n_exps))

function fairfunc(exp,n)
  [rand(0:1) for i in 1:n]
end
m_fair = Model("fair", (exp) -> fairfunc(exp,n_exps))

function markovprob(exp,data)
  transitions = 0
  for i in 1:3
    if exp[i] != exp[i+1]
      transitions += 1
    end
  end
  if transitions == 3
    t_prob = 4/5
  elseif transitions == 2
    t_prob = 3/5
  elseif transitions == 1
    t_prob = 2/5
  elseif transitions == 0
    t_prob = 1/5
  end
  if exp[4] == 1
    p_head = 1 - t_prob
  else
    p_head = t_prob
  end
  heads = sum(data)
  p_head^heads * (1 - p_head)^(length(data) - heads)
end
#probability of data under fair model
function fairprob(exp,data)
  0.5 ^ (length(data))
end
function biasedprob(exp,data)
  if sum(exp) == 4
    p_head = 5/6
  elseif sum(exp) == 3
    p_head = 2/3
  elseif sum(exp) == 2
    p_head = 1/2
  elseif sum(exp) == 1
    p_head = 1/3
  else
    p_head = 1/6
  end
  heads = sum(data)
  p_head^heads * (1 - p_head)^(length(data) - heads)
end
function coinposterior(prior,sequence)
  newpost = copy(prior)
  likelihoods = [fairprob, biasedprob, markovprob]
  for (exp,data) in sequence
    newpost = exactpost(prior,likelihoods,exp,data)
  end
  newpost
end
function coinoptimalseq(prior,sequence,samples)
  newprior = coinposterior(prior, sequence)
  #println(newprior)
  coinoptimal(newprior,samples)
end
#outputs best exp given belief and sample size
function coinoptimal(prior,samples)
  models = [m_fair,m_biased,m_markov]
  weights = ProbabilityWeights(prior)
  likelihoods = [fairprob, biasedprob, markovprob]
  #println(newprior)
  experiments = [[1,1,1,1],[1,1,1,0],[1,1,0,1],[1,1,0,0],[1,0,1,1],[1,0,1,0],
  [1,0,0,1],[1,0,0,0],[0,1,1,1],[0,1,1,0],[0,1,0,1],[0,1,0,0],[0,0,1,1],[0,0,1,0],
  [0,0,0,1],[0,0,0,0]]
  analyticoptimalexp(prior,() -> sample(models,weights),likelihoods,experiments,samples)
end


coinoptimal (generic function with 1 method)

In [14]:
struct Model
  name
  func
end
struct ZeroProb <: Exception
end
function analyticoptimalexp(prior,priorsamp,likelihoods,experiments,samples)
  max = 0
  bestexp = -1
  for exp in experiments
    u = analyticutil(prior,priorsamp,likelihoods,exp,samples)
    #println(exp,u)
    if u >= max
      max = u
      bestexp = exp
    end
  end
  bestexp
end

analyticoptimalexp (generic function with 1 method)

In [15]:
using Test
#all experiment sequences
experiments = [[1,1,1,1],[1,1,1,0],[1,1,0,1],[1,1,0,0],[1,0,1,1],[1,0,1,0],
[1,0,0,1],[1,0,0,0],[0,1,1,1],[0,1,1,0],[0,1,0,1],[0,1,0,0],[0,0,1,1],[0,0,1,0],
[0,0,0,1],[0,0,0,0]]
n_exps=1
"""
biased model, from paper
"""
function biasedfunc(exp,n)
  if sum(exp) == 4
    phead = 5/6
  elseif sum(exp) == 3
    phead = 2/3
  elseif sum(exp) == 2
    phead = 1/2
  elseif sum(exp) == 1
    phead = 1/3
  else
    phead = 1/6
  end
  [Int(rand() < phead) for i in 1:n]
end
m_biased = Model("biased",(exp) -> biasedfunc(exp,n_exps))
"""
markov model, from paper
"""
function markovfunc(exp,n)
  transitions = 0
  for i in 1:3
    if exp[i] != exp[i+1]
      transitions += 1
    end
  end
  if transitions == 3
    t_prob = 4/5
  elseif transitions == 2
    t_prob = 3/5
  elseif transitions == 1
    t_prob = 2/5
  elseif transitions == 0
    t_prob = 1/5
  end
  if rand() < t_prob
    1 - exp[4]
  else
    exp[4]
  end
  vals=[]
  for i in 1:n
    if rand() < t_prob
      push!(vals, 1 - exp[4])
    else
      push!(vals, exp[4])
    end
  end
  vals
end
m_markov = Model("markov",(exp) -> markovfunc(exp,n_exps))
"""
fair model
"""
function fairfunc(exp,n)
  [rand(0:1) for i in 1:n]
end
m_fair = Model("fair", (exp) -> fairfunc(exp,n_exps))

"""
uniform prior samplers
"""
function allpriorsamp()
  rand([m_fair,m_biased,m_markov])
end
function fairmarkovsamp()
  rand([m_fair,m_markov])
end
function markovbiasedsamp()
  rand([m_markov,m_biased])
end
function fairbiasedsamp()
  rand([m_fair,m_biased])
end
#@test optimalexp(allpriorsamp,experiments) in [[0,0,0,0],[1,1,1,1]]
#@test optimalexp(markovbiasedsamp,experiments) in [[1,0,1,0],[0,1,0,1]]
"""
faster special case of coin
"""
#probability of experiment under markov model
function markovprob(exp,data)
  transitions = 0
  for i in 1:3
    if exp[i] != exp[i+1]
      transitions += 1
    end
  end
  if transitions == 3
    t_prob = 4/5
  elseif transitions == 2
    t_prob = 3/5
  elseif transitions == 1
    t_prob = 2/5
  elseif transitions == 0
    t_prob = 1/5
  end
  if exp[4] == 1
    p_head = 1 - t_prob
  else
    p_head = t_prob
  end
  heads = sum(data)
  p_head^heads * (1 - p_head)^(length(data) - heads)
end
#probability of data under fair model
function fairprob(exp,data)
  0.5 ^ (length(data))
end
function biasedprob(exp,data)
  if sum(exp) == 4
    p_head = 5/6
  elseif sum(exp) == 3
    p_head = 2/3
  elseif sum(exp) == 2
    p_head = 1/2
  elseif sum(exp) == 1
    p_head = 1/3
  else
    p_head = 1/6
  end
  heads = sum(data)
  p_head^heads * (1 - p_head)^(length(data) - heads)
end
likelihoods = [fairprob, biasedprob, markovprob]
println(analyticoptimalexp([1/3,1/3,1/3],allpriorsamp,likelihoods,experiments, 100000))
println(analyticoptimalexp([1/2,1/2],markovbiasedsamp,[markovprob,biasedprob],experiments, 100000))
println(coinoptimalseq([1/3,1/3,1/3],[((1,1,0,0),1),((0,1,0,0),0)],10000))
println(coinoptimal([1/3,1/3,1/3],10000))

[1, 1, 1, 1]
[1, 0, 1, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]


In [ ]:

function gen_prior_data(n_samples,3)
   priors=zeros(Float64,(3,n_samples))
    for idx in 1:n_samples
        first_prior=rand(Uniform(0.0,0.99),1)[1]
        second_prior=rand(Uniform(0.0,1-first_prior),1)[1]
        third_prior=1.0-first_prior-second_prior
        prior=[first_prior,second_prior,third_prior]
        priors[:,idx]=prior
    end 
    
    for prior in eachcol(priors)
        
        
    end
end

In [40]:
experiment_to_int=Dict(
    "1111"=>1, 
    "1110" => 2 , 
    "1101" => 3,
    "1011"=> 4,
    "0111" => 5,
    "1100" => 6,
    "0011" => 7,
    "1010" => 8,
    "0101" => 9,
    "1001" => 10,
    "0110" => 11,
    "0001" => 12,
    "0010" => 13,
    "0100" => 14,
    "1000" => 15,
    "0000" => 0
)
function generate_custom_data(n_classes,OED,n_samples,epochs,sample_size)
    priors=zeros(Float64,(3,n_samples))
    for idx in 1:n_samples
        first_prior=rand(Uniform(0.0,0.99),1)[1]
        second_prior=rand(Uniform(0.0,1-first_prior),1)[1]
        third_prior=1.0-first_prior-second_prior
        prior=[first_prior,second_prior,third_prior]
        priors[:,idx]=prior
    end
    labels=Any[]
    for prior in eachcol(priors)
        exp=OED(prior,sample_size)
        sequence=""
        for num in exp
            sequence=sequence*string(num)
        end
        push!(labels,experiment_to_int[sequence])
    end
    Y=onehotbatch(labels,0:n_classes-1)|> gpu
    X=priors |> gpu
    return repeated((X,Y),epochs),X,Y
end


generate_custom_data (generic function with 2 methods)

In [41]:
train_dataset,x_train,y_train=generate_custom_data(16,coinoptimal,5000,100,1000)

(Base.Iterators.Take{Base.Iterators.Repeated{Tuple{Array{Float64,2},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}}}}(Base.Iterators.Repeated{Tuple{Array{Float64,2},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}}}(([0.19961920766104516 0.7801702986626042 … 0.878325211147257 0.4686984528577132; 0.28001625527137236 0.0020810042563720803 … 0.0274089469668254 0.279141946950645; 0.5203645370675825 0.21774869708102368 … 0.09426584188591756 0.25215960019164185], Bool[0 0 … 1 0; 0 1 … 0 1; … ; 0 0 … 0 0; 0 0 … 0 0])), 100), [0.19961920766104516 0.7801702986626042 … 0.878325211147257 0.4686984528577132; 0.28001625527137236 0.0020810042563720803 … 0.0274089469668254 0.279141946950645; 0.5203645370675825 0.21774869708102368 … 0.09426584188591756 0.25215960019164185], Bool[0 0 … 1 0; 0 1 … 0 1; … ; 0 0 … 0 0; 0 0 … 0 0])

In [42]:
test_dataset,x_test,y_test=generate_custom_data(16,coinoptimal,100,1,1000)

(Base.Iterators.Take{Base.Iterators.Repeated{Tuple{Array{Float64,2},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}}}}(Base.Iterators.Repeated{Tuple{Array{Float64,2},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}}}(([0.7293884404413752 0.9728108072926395 … 0.7153194652975476 0.8348218765714513; 0.043183392995129445 0.008119698958000597 … 0.2845015606094253 0.09456202795899782; 0.22742816656349538 0.019069493749359935 … 0.00017897409302708445 0.07061609546955083], Bool[1 1 … 0 0; 0 0 … 1 1; … ; 0 0 … 0 0; 0 0 … 0 0])), 1), [0.7293884404413752 0.9728108072926395 … 0.7153194652975476 0.8348218765714513; 0.043183392995129445 0.008119698958000597 … 0.2845015606094253 0.09456202795899782; 0.22742816656349538 0.019069493749359935 … 0.00017897409302708445 0.07061609546955083], Bool[1 1 … 0 0; 0 0 … 1 1; … ; 0 0 … 0 0; 0 0 … 0 0])

In [44]:
model=Chain(
Dense(3,16,relu),
Dense(16,32,relu),
Dense(32,16,relu),
    softmax)|> gpu

loss(x, y) = crossentropy(model(x), y)
accuracy(x, y) = mean(onecold(cpu(model(x))) .== onecold(cpu(y)))

# train_dataset,train_x,train_y=generate_data(8000,500)
# test_dataset,test_x,test_y=generate_data(100,1)
evalcb=() -> @show (loss(x_train,y_train))
opt=ADAM()

ps=Flux.params(model)
Flux.train!(loss, ps, train_dataset, opt, cb = throttle(evalcb, 10))
println("Training accuracy:")
@show accuracy(x_train, y_train)
println("Testing accuracy:")
@show accuracy(x_test, y_test)

loss(x_train, y_train) = 2.7812874f0
Training accuracy:
accuracy(x_train, y_train) = 0.4226
Testing accuracy:
accuracy(x_test, y_test) = 0.45


0.45

In [30]:
sequence=""
for num in test
    sequence=sequence*string(num)
end

In [31]:
sequence

"1111"